In [1]:
import credentials # Api key is stored in this file, remove to avoid errors if you clone from github

import pvdeg
import pvlib
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from scipy.linalg import cholesky
# from math import e as euler

First get weather data and metadata for a desired location (latitude and logitude)

In [2]:
# change to desired values (currently Miami)
latitude = 25.783388
longitude = -80.189029

API_KEY = credentials.API_KEY # my personal NREL api key
email ='tobin.ford@nrel.gov' # replace these values with your appropriate information and remove or comment out first line of first block (import credentials)

# reads NSRDB data 
weather_df, meta = pvlib.iotools.get_psm3(latitude, longitude, API_KEY, email, names='2019', map_variables=True)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pvlib/iotools/psm3.py:183: pvlibDeprecationWarning: The ``get_psm3`` function will default to leap_day=True starting in pvlib 0.11.0. Specify leap_day=True to enable this behavior now, or specify leap_day=False to hide this warning.
  warnings.warn(


User has 3 parameters for initial implementation: See Kempe's "Deg Miami" tab in excel<br>



activation energy, Ea <br>
irradiance relation, x<br>
ln(R0)<br>

|           |   Ea   |   x   | ln(R0) |
|:---------:|:-----:|:----:|:-------:|
|   **Ea**  |   1   |   a  |   b     |
|   **x**   |   a   |   1  |   c     |
| **ln(R0)**|   b   |   c  |   1     |

Notice symmetry across diagonal <br>

In [3]:
# USER ENTERED VALUES
# Correlation Coefficients
Ea_X = 0.0269
Ea_lnR0 = -0.9995
X_lnR0 = -0.0400

# Activation Energy
mean_Ea = 129 # average
sd_Ea = 3.4 # standard deviation

# Irradiance relation
mean_X = 0.0341 # average
sd_X = 0.0992757 # standard deviation

# ln(R0)
mean_R0 = 13.7223084 
sd_R0 = 2.47334772

# number of iterations
n = 20000


In [4]:
# notice symmetry of matrix
A = np.array([[1, Ea_X, Ea_lnR0],
              [Ea_X, 1, X_lnR0],
              [Ea_lnR0, X_lnR0, 1]])

# conceptually similar to the square root of a matrix
A_decomp = cholesky(A, lower=True)

# generating standard distribution of points for each
Ea, X, R0 = np.random.normal(mean_Ea, sd_Ea, 20_000), np.random.normal(mean_X, sd_X, 20_000), np.random.normal(mean_R0, sd_R0, 20_000)

# transpose built in so I don't have to do it with another function
samples_matrix = np.array([Ea, X, R0])



In [5]:
# correlated stats pre-input to function

# not entirely sure what I should do here,
# do i multiply the transpose by the lower cholesky to get correlated input data for the equation on the spreadsheet
# using MonteCarloEaLnRoX

# this gives us correlated input samples
# looks like what kempe's code was doing in "MonteCarloEaLnRoX" but it is hard to tell what's happening in his code
correlated_samples = np.matmul(A_decomp, samples_matrix)

correlated_df = pd.DataFrame(correlated_samples.T, columns=['Ea', 'X', 'R0'])

# is back tracking on by default
sol_pos = pvdeg.spectral.solar_position(weather_df, meta)
poa_irradiance = pvdeg.spectral.poa_irradiance(weather_df, meta)
temp_mod = pvdeg.temperature.module(weather_df=weather_df, meta=meta, poa=poa_irradiance, conf='open_rack_glass_polymer')
# we only care about global irradiance in this case


In [51]:
correlated_df

,Ea,X,R0
0,126.157720,3.504130,-125.675674
1,128.069313,3.438949,-127.650548
2,128.297034,3.601389,-127.841737
3,123.476940,3.385414,-123.053608
4,126.842607,3.481976,-126.496209
...,...,...,...
19995,136.075105,3.764268,-135.435331
19996,132.888935,3.652042,-132.473234
19997,131.975698,3.621940,-131.486187
19998,128.794528,3.616764,-128.309447


In [45]:
def mikeArrhenius(weather_df, samples):
# Ea1 = Ea(Z) / 8.31446261815324E-03 'This reduces the amount of calculations in the loop
# Degradation(Z) = Degradation(Z) + Exp(LnRo(Z)) * Exp(-Ea1 / (273.15 + T(Y, 1))) * ((Irr(Y, 1) / 1000) ^ X(Z))

    # FUNCTION for x,y,z in zip(correlated_df['X'], correlated_df['Ea'], corrleated_df['R0'])
    d = np.exp(samples['R0']) * np.exp((samples['Ea'] / 8.31446261815324E-03) / (273.15 + weather_df['temp_air'])) * (weather_df['ghi'] / 1000 ** samples['X'])
    
    return pd.DataFrame(d)

In [46]:
def mikeArrhenius2(weather_df, samples): # not using argument properly, it didnt like when i did this

    #['Ea', 'X', 'R0']
    d = [(np.exp(z) * np.exp((x / 8.31446261815324E-03) / (273.15 + weather_df['temp_air'])) * (weather_df['ghi'] / 1000 ** y)).mean() for x, y, z in zip(samples['Ea'], samples['X'], samples['R0'])] 
    # added .mean(), is this right

    return pd.DataFrame(d)

In [47]:
temp1 = mikeArrhenius(weather_df=weather_df, samples=correlated_df.iloc[0]) # works mostly as desired

#temp2 = mikeArrhenius2(weather_df=weather_df, samples=correlated_df.iloc[0]) # gets mad if i pass as value
temp2 = mikeArrhenius2(weather_df=weather_df, samples=correlated_df) # gets mad if i pass as value

# difference is probably something about how I am passing values to the functions,
# the original function takes data as arguments but the second just pulls them from globalscope, this could cause undesired behavior

In [48]:
temp1.head(20)

,0
2019-01-01 00:30:00-05:00,0.000000e+00
2019-01-01 01:30:00-05:00,0.000000e+00
2019-01-01 02:30:00-05:00,0.000000e+00
2019-01-01 03:30:00-05:00,0.000000e+00
2019-01-01 04:30:00-05:00,0.000000e+00
2019-01-01 05:30:00-05:00,0.000000e+00
2019-01-01 06:30:00-05:00,0.000000e+00
2019-01-01 07:30:00-05:00,5.006961e-42
2019-01-01 08:30:00-05:00,1.646902e-41
2019-01-01 09:30:00-05:00,3.816648e-41


In [50]:

temp2.head(20)
# what are the units on this 
# these are significantly different than the calculated rates on deg miami tab n*10^-a
# ex 5.2*10^-6
# where could the error be coming from 

,0
0,1.697827e-41
1,7.983182e-42
2,2.353238e-42
3,1.802514e-40
4,1.147655e-41
5,1.804154e-41
6,8.019067e-43
7,2.841593e-43
8,2.760238e-42
9,3.329113e-42
